In [67]:
import numpy as np
import math
import sys
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr
from scipy.stats import pearsonr

# 1. Compute the similarity between the different embedding files

### 1.1 Retrieve the necessary embeddings and lexicon

In [68]:
''' Read all the word vectors and normalize them '''
def read_word_vecs(filename):
  """
  - input: name of the file containing the word vectors
  """
  wordVectors = {}
  with open(filename, 'r', encoding='utf-8') as fileObject:
    first_line = True
    for line in fileObject:
      line = line.strip().lower()
      # Skip the first line
      if first_line:
        first_line =False
        continue
      # The first word is assumed to be the word itself, and the remaining words are assumed to be the components of the word vector
      word = line.split()[0]
      # initialize a numpy array of zeros with the same length as the word vector
      wordVectors[word] = np.zeros(len(line.split())-1, dtype=float)
      for index, vecVal in enumerate(line.split()[1:]):
        # assign the values in the numpy array to the corresponding components of the word vector
        wordVectors[word][index] = float(vecVal)
      ''' normalize weight vector '''
      # divide each element by the square root of the sum of the squares of all the elements in the array
      # plus a small constant (1e-6) to avoid division by zero
      wordVectors[word] /= math.sqrt((wordVectors[word]**2).sum() + 1e-6)
  
  # standard error indicating that the vectors have been read from the file 
  sys.stderr.write("Vectors read from: "+filename+" \n")
  return wordVectors

In [69]:
# Load pre_trained word embeddings
# ENGLISH
EN_file_non_ret = read_word_vecs("../data/English/wordEmbeddings/vectors_datatxt_250_sg_w10_i5_c500_gensim_clean")
# FRENCH
FR_file_non_ret_50 = read_word_vecs("../data/French/word_embeddings/vecs50-linear-frwiki")
FR_file_non_ret_100 = read_word_vecs("../data/French/word_embeddings/vecs100-linear-frwiki")

# Load retrofitted word embeddings
# ENGLISH
EN_file_ret =read_word_vecs("../data/English/output_vectors/output_vectors.txt")
# FRENCH
FR_file_ret_100 = read_word_vecs("../data/French/output_vectors/output_vectors.txt")

Vectors read from: ../data/English/wordEmbeddings/vectors_datatxt_250_sg_w10_i5_c500_gensim_clean 
Vectors read from: ../data/French/word_embeddings/vecs50-linear-frwiki 
Vectors read from: ../data/French/word_embeddings/vecs100-linear-frwiki 
Vectors read from: ../data/English/output_vectors/output_vectors.txt 
Vectors read from: ../data/French/output_vectors/output_vectors.txt 


In [70]:
#ENGLISH

#open english lexical_similarity file
lex_sim_eng = open ("../data/English/lexicon/ws353_lexical_similarity.txt", 'r')
#list of word pairs from the lexical similarity file
word_pairs_eng = []
#list of human scores for each word pair
hum_score_eng = []

#for every line in the file
for line in lex_sim_eng:
    #split the line by space
    #one line contains the word pair and its score
    w1, w2, score = line.split()
    #add the words to word_pairs
    word_pairs_eng.append((w1, w2))
    #add the ratings to human_score
    hum_score_eng.append(score)

In [71]:
print(word_pairs_eng[0])
print(hum_score_eng[0])

('love', 'sex')
6.77


In [72]:
#FRENCH

#open lexical_similarity file
lex_sim_fre = open ("../data/French/lexicon/rg65_french.txt", 'r')
#list of word pairs from the lexical similarity file
word_pairs_fre = []
#list of human scores for each word pair
hum_score_fre = []
#open lexical_similarity file

#for every line in the file
for line in lex_sim_fre:
    #split the line by space
    #one line contains the word pair and its score
    w1, w2, score = line.split()
    #add the words to word_pairs
    word_pairs_fre.append((w1, w2))
    #add the ratings to human_score
    hum_score_fre.append(score)

In [73]:
print(word_pairs_fre[0])
print(hum_score_fre[0])

('corde', 'sourire')
0.00


### 1.2 Compute cosine similarity for non-retrofitted vectors

In [74]:
#ENGLISH

#list of cosine similarities between non-retrofitted vectors
cos_non_ret_eng = []

#for every word pair
for w1, w2 in word_pairs_eng:
    #check if the words exist in the word_embeddings
    if w1 in EN_file_non_ret and w2 in EN_file_non_ret:
        #retrieve word1's vector 
        vec1 = EN_file_non_ret[w1]
        #retreive word2's vector
        vec2 = EN_file_non_ret[w2]
        #calculate cosine between the two vectors
        similarity = 1 - cosine(vec1, vec2)
        #add the result to scores
        cos_non_ret_eng.append(round(similarity, 2))
    #otherwise if words don't exist in the word embeddings
    else:
        #assign default value of 0.0
        cos_non_ret_eng.append(0.00)        

In [75]:
#FRENCH

#list of cosine similarities between non-retrofitted vectors
cos_non_ret_fre_100 = []

#for every word pair
for w1, w2 in word_pairs_fre:
    #check if the words exist in the word_embeddings
    if w1 in FR_file_non_ret_100 and w2 in FR_file_non_ret_100:
        #retrieve word1's vector 
        vec1 = FR_file_non_ret_100[w1]
        #retreive word2's vector
        vec2 = FR_file_non_ret_100[w2]
        #calculate cosine between the two vectors
        similarity = 1 - cosine(vec1, vec2)
        #add the result to scores
        cos_non_ret_fre_100.append(round(similarity, 2))
    #otherwise if words don't exist in the word embeddings
    else:
        #assign default value of 0.0
        cos_non_ret_fre_100.append(0.00)        

In [76]:
print(hum_score_eng)
print(cos_non_ret_eng)

['6.77', '7.35', '10.00', '7.46', '7.62', '7.58', '5.77', '6.31', '7.50', '6.77', '7.42', '6.85', '6.19', '5.92', '7.00', '6.62', '6.81', '4.62', '5.81', '7.08', '8.08', '1.62', '1.31', '0.92', '1.81', '6.69', '3.73', '0.92', '7.46', '8.12', '7.73', '9.15', '0.31', '0.23', '8.58', '5.92', '6.69', '8.46', '7.65', '1.62', '9.44', '8.62', '9.03', '6.81', '6.63', '7.56', '6.73', '7.65', '2.50', '8.38', '7.38', '6.19', '6.73', '7.92', '8.12', '7.35', '4.88', '5.54', '8.46', '8.13', '3.04', '1.31', '5.96', '6.87', '7.85', '2.65', '8.94', '8.96', '9.29', '8.83', '9.10', '8.87', '9.02', '9.29', '8.79', '7.52', '7.10', '7.38', '6.46', '6.27', '2.69', '4.46', '5.85', '5.00', '2.08', '4.42', '4.38', '1.85', '3.08', '0.92', '3.15', '0.92', '0.54', '2.08', '0.54', '0.62', '8.42', '9.08', '9.04', '8.27', '7.57', '7.29', '8.50', '7.73', '6.88', '5.65', '3.31', '8.00', '8.00', '7.08', '6.85', '7.00', '4.77', '5.62', '5.87', '8.08', '7.00', '6.85', '7.42', '6.58', '6.42', '8.21', '7.69', '7.23', '6.71'

In [78]:
print(hum_score_fre)
print(cos_non_ret_fre_100)

['0.00', '0.00', '0.06', '0.11', '0.00', '0.00', '0.06', '0.00', '0.00', '0.22', '0.22', '0.44', '0.06', '0.17', '0.17', '0.50', '0.22', '0.11', '0.33', '0.39', '0.29', '0.06', '0.17', '0.44', '0.17', '0.61', '0.11', '0.28', '0.06', '2.17', '0.56', '0.28', '0.44', '0.33', '0.22', '0.56', '0.56', '0.94', '2.00', '0.83', '1.28', '1.65', '2.41', '2.78', '2.89', '3.28', '2.78', '2.67', '2.94', '3.33', '3.39', '1.50', '1.89', '2.59', '3.56', '2.50', '3.72', '3.00', '4.00', '3.83', '3.00', '4.00', '3.94', '3.22', '2.17']
[0.47, 0.18, 0.4, 0.2, 0.24, 0.28, 0.37, 0.08, 0.35, 0.44, 0.22, 0.56, 0.35, 0.19, 0.27, 0.51, 0.44, 0.3, 0.54, 0.33, 0.1, 0.2, 0.17, 0.5, 0.38, 0.34, 0.44, 0.23, 0.4, 0.31, 0.11, 0.45, 0.48, 0.21, 0.26, 0.51, 0.32, 0.31, 0.46, 0.21, 0.16, 0.43, 0.5, 0.4, 0.64, 0.53, 0.36, 0.71, 0.19, 0.51, 0.32, 0.44, 0.33, 0.55, 0.28, 0.25, 0.67, 0.57, 1, 0.55, 0.6, 1, 0.51, 0.46, 0.02]


### 1.3 Compute cosine similarity for retrofitted vectors

In [83]:
#ENGLISH

#list of cosine similarities between retrofitted vectors
cos_ret_eng = []

#for every word pair
for w1, w2 in word_pairs_eng:
    #check if the words exist in the word_embeddings
    if w1 in EN_file_ret and w2 in EN_file_ret:
        #retrieve word1's vector 
        vec1 = EN_file_ret[w1]
        #retreive word2's vector
        vec2 = EN_file_ret[w2]
        #calculate cosine between the two vectors
        similarity = 1 - cosine(vec1, vec2)
        #add the result to scores
        cos_ret_eng.append(round(similarity, 2))
    #otherwise if words don't exist in the word embeddings
    else:
        #assign default value of 0.0
        cos_ret_eng.append(0.00)        

In [84]:
#FRENCH

#list of cosine similarities between retrofitted vectors
cos_ret_fre_100 = []

#for every word pair
for w1, w2 in word_pairs_fre:
    #check if the words exist in the word_embeddings
    if w1 in FR_file_ret_100 and w2 in FR_file_ret_100:
        #retrieve word1's vector 
        vec1 = FR_file_ret_100[w1]
        #retreive word2's vector
        vec2 = FR_file_ret_100[w2]
        #calculate cosine between the two vectors
        similarity = 1 - cosine(vec1, vec2)
        #add the result to scores
        cos_ret_fre_100.append(round(similarity, 2))
    #otherwise if words don't exist in the word embeddings
    else:
        #assign default value of 0.0
        cos_ret_fre_100.append(0.00)        

In [85]:
print(hum_score_eng)
print(cos_ret_eng)

['6.77', '7.35', '10.00', '7.46', '7.62', '7.58', '5.77', '6.31', '7.50', '6.77', '7.42', '6.85', '6.19', '5.92', '7.00', '6.62', '6.81', '4.62', '5.81', '7.08', '8.08', '1.62', '1.31', '0.92', '1.81', '6.69', '3.73', '0.92', '7.46', '8.12', '7.73', '9.15', '0.31', '0.23', '8.58', '5.92', '6.69', '8.46', '7.65', '1.62', '9.44', '8.62', '9.03', '6.81', '6.63', '7.56', '6.73', '7.65', '2.50', '8.38', '7.38', '6.19', '6.73', '7.92', '8.12', '7.35', '4.88', '5.54', '8.46', '8.13', '3.04', '1.31', '5.96', '6.87', '7.85', '2.65', '8.94', '8.96', '9.29', '8.83', '9.10', '8.87', '9.02', '9.29', '8.79', '7.52', '7.10', '7.38', '6.46', '6.27', '2.69', '4.46', '5.85', '5.00', '2.08', '4.42', '4.38', '1.85', '3.08', '0.92', '3.15', '0.92', '0.54', '2.08', '0.54', '0.62', '8.42', '9.08', '9.04', '8.27', '7.57', '7.29', '8.50', '7.73', '6.88', '5.65', '3.31', '8.00', '8.00', '7.08', '6.85', '7.00', '4.77', '5.62', '5.87', '8.08', '7.00', '6.85', '7.42', '6.58', '6.42', '8.21', '7.69', '7.23', '6.71'

In [86]:
print(hum_score_fre)
print(cos_ret_fre_100)

['0.00', '0.00', '0.06', '0.11', '0.00', '0.00', '0.06', '0.00', '0.00', '0.22', '0.22', '0.44', '0.06', '0.17', '0.17', '0.50', '0.22', '0.11', '0.33', '0.39', '0.29', '0.06', '0.17', '0.44', '0.17', '0.61', '0.11', '0.28', '0.06', '2.17', '0.56', '0.28', '0.44', '0.33', '0.22', '0.56', '0.56', '0.94', '2.00', '0.83', '1.28', '1.65', '2.41', '2.78', '2.89', '3.28', '2.78', '2.67', '2.94', '3.33', '3.39', '1.50', '1.89', '2.59', '3.56', '2.50', '3.72', '3.00', '4.00', '3.83', '3.00', '4.00', '3.94', '3.22', '2.17']
[0.47, 0.18, 0.4, 0.2, 0.24, 0.37, 0.31, 0.36, 0.47, 0.27, 0.45, 0.56, 0.35, 0.19, 0.27, 0.24, 0.44, 0.12, 0.54, 0.31, 0.15, 0.32, 0.17, 0.38, 0.38, 0.34, 0.44, 0.34, 0.4, 0.31, 0.17, 0.45, 0.48, 0.53, 0.26, 0.51, 0.57, 0.19, 0.39, 0.12, 0.2, 0.43, 0.5, 0.4, 0.64, 0.42, 0.29, 0.71, 0.19, 0.51, 0.35, 0.23, 0.33, 0.22, 0.16, 0.25, 0.67, 0.35, 1, 0.26, 0.6, 1, 0.9, 0.46, 0.02]


# 2. Computing correlation scores

### 2.1 Spearman

In [89]:
#calculate the spearman rank correlation between human ratings and non-retrofitted vectors
spearman_non_ret_eng, _ = spearmanr(hum_score_eng, cos_non_ret_eng)
print("The Spearman correlation coefficient for english (using non-retrofitted vectors): ", round(spearman_non_ret_eng,3))

print("")

spearman_non_ret_fre_100, _ = spearmanr(hum_score_fre, cos_non_ret_fre_100)
print("The Spearman correlation coefficient for french (using non-retrofitted vectors (100)): ", round(spearman_non_ret_fre_100,3))

The Spearman correlation coefficient for english (using non-retrofitted vectors):  0.576

The Spearman correlation coefficient for french (using non-retrofitted vectors (100)):  0.472


In [90]:
#calculate the spearman rank correlation between human ratings and retrofitted vectors
spearman_ret_eng, _ = spearmanr(hum_score_eng, cos_ret_eng)
print("The Spearman correlation coefficient for english (using retrofitted vectors): ", round(spearman_ret_eng,3))

print("")

spearman_ret_fre_100, _ = spearmanr(hum_score_fre, cos_ret_fre_100)
print("The Spearman correlation coefficient for french (using retrofitted vectors): ", round(spearman_ret_fre_100,3))

The Spearman correlation coefficient for english (using retrofitted vectors):  0.137

The Spearman correlation coefficient for french (using retrofitted vectors):  0.226


### 2.2 Pearson

In [91]:
#convert values from string to float
hum_score_eng = [float(value) for value in (hum_score_eng)]
#compute the pearson correlation coefficient and the p_value between human ratings and non-retrofitted vectors
pearson_non_ret_eng, p_value_non_ret_eng = pearsonr(hum_score_eng, cos_non_ret_eng)

#print results
print("The Pearson correlation coefficient for english (using non-retrofitted vectors : ", round(pearson_non_ret_eng, 4))
print("p_value: ", round(p_value_non_ret_eng, 4))

print("")

#convert values from string to float
hum_score_fre = [float(value) for value in (hum_score_fre)]
#compute the pearson correlation coefficient and the p_value
pearson_non_ret_fre_100, p_value_non_ret_fre_100 = pearsonr(hum_score_fre, cos_non_ret_fre_100)

print("")
#print results
print("The Pearson correlation coefficient for french (using non-retrofitted vectors (100)): ", round(pearson_non_ret_fre_100, 4))
print("p_value: ", round(p_value_non_ret_fre_100, 4))


The Pearson correlation coefficient for english (using non-retrofitted vectors :  0.5606
p_value:  0.0


The Pearson correlation coefficient for french (using non-retrofitted vectors (100)):  0.5209
p_value:  0.0


In [92]:
#compute the pearson correlation coefficient and the p_value between human ratings and retrofitted vectors
pearson_ret_eng, p_value_ret_eng = pearsonr(hum_score_eng, cos_ret_eng)

#print results
print("The Pearson correlation coefficient for english (using retrofitted vectors : ", round(pearson_ret_eng, 4))
print("p_value: ", round(p_value_ret_eng, 4))

pearson_ret_fre_100, p_value_ret_fre_100 = pearsonr(hum_score_fre, cos_ret_fre_100)

#print results
print("The Pearson correlation coefficient for french (using retrofitted vectors: ", round(pearson_ret_fre_100, 4))
print("p_value: ", round(p_value_ret_fre_100, 4))


The Pearson correlation coefficient for english (using retrofitted vectors :  0.1908
p_value:  0.0003
The Pearson correlation coefficient for french (using retrofitted vectors:  0.3891
p_value:  0.0014
